In [111]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
import io
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from pyathena import connect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, \
RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, mean_squared_error,\
precision_score, recall_score, f1_score, accuracy_score,\
confusion_matrix, plot_confusion_matrix, classification_report
from sagemaker.tuner import HyperparameterTuner
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import loguniform
import warnings
warnings.filterwarnings('ignore')
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [112]:
ingest_create_athena_db_passed = False

In [113]:
database_name = "ecommerce"

In [114]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [115]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [116]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS ecommerce


""


In [117]:
#S3 bucket name: olistteam5
#S3 URL: s3://olistteam5/ecommerce/

In [118]:
olist_dir='s3://olistteam5/ecommerce/olist_order_items_dataset.csv'

In [119]:
table_name ='olist_order_items_dataset'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                'order_id' string,
                'order_item_id' int,
                'product_id' string,
                'seller_id' string,
                'shipping_limit_date' date,
                'price' int,
                'freight_value' int,
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
print(create_table)



    CREATE EXTERNAL TABLE IF NOT EXISTS ecommerce.olist_order_items_dataset(
                'order_id' string,
                'order_item_id' int,
                'product_id' string,
                'seller_id' string,
                'shipping_limit_date' date,
                'price' int,
                'freight_value' int,
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION 's3://olistteam5/ecommerce/olist_order_items_dataset.csv/olist_order_items_dataset'
                TBLPROPERTIES ('skip.header.line.count'='1')



In [110]:
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

Failed to execute query.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/pyathena/common.py", line 305, in _execute
    **request
  File "/opt/conda/lib/python3.7/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 379, in __call__
    do = self.iter(retry_state=retry_state)
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 314, in iter
    return fut.result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 428, in result
    return self.__get_result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 384, in __get_result
    raise self._exception
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 382, in __call__
    result = fn(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 530, in _api_call
    return

DatabaseError: Execution failed on sql: 
    CREATE EXTERNAL TABLE IF NOT EXISTS ecommerce.olist_order_items_dataset(
                'order_id' string,
                'order_item_id' int,
                'product_id' string,
                'seller_id' string,
                'shipping_limit_date' date,
                'price' int,
                'freight_value' int,
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION 's3://olistteam5/ecommerce/olist_order_items_dataset.csv/olist_order_items_dataset'
                TBLPROPERTIES ('skip.header.line.count'='1')

An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 1:8: mismatched input 'EXTERNAL'. Expecting: 'OR', 'SCHEMA', 'TABLE', 'VIEW'
unable to rollback